In [2]:
# Importing necessary libraries
import numpy as np
import splitfolders
import tensorflow as tf

from tensorflow import keras
from keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import confusion_matrix, classification_report

In [3]:
# Splitting data into train, test, and validation sets
tb_dir = "data/TB_Chest_Radiography_Database"
splitfolders.ratio(tb_dir, output="data/Split_TB_Data", seed=120, ratio=(0.7, 0.15, 0.15), group_prefix=None)

Copying files: 0 files [00:00, ? files/s]

Copying files: 4200 files [00:02, 1440.53 files/s]


In [4]:
# Data preprocessing and augmentation
datagen = ImageDataGenerator(rescale=1./255,
                             shear_range=0.2,
                             zoom_range=0.2,
                             horizontal_flip=True)

In [5]:
IMG_SIZE = 224
split_tb_dir = "data/Split_TB_Data"

In [6]:
train_generator = datagen.flow_from_directory(
    "data/Split_TB_Data/train",
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=16,
    class_mode='categorical'
)

test_generator = datagen.flow_from_directory(
    "data/Split_TB_Data/test",
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=16,
    class_mode='categorical'
)

val_generator = datagen.flow_from_directory(
    "data/Split_TB_Data/val",
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=16,
    class_mode='categorical'
)

Found 3797 images belonging to 2 classes.
Found 1159 images belonging to 2 classes.
Found 1166 images belonging to 2 classes.


In [7]:
# Building the model
# normalizer = keras.layers.Rescaling(scale=1/255)

In [8]:
dense_net = keras.applications.DenseNet169(weights='imagenet', 
                                           input_shape=(IMG_SIZE, IMG_SIZE, 3), 
                                           include_top=False)

for layer in dense_net.layers:
    layer.trainable = False

2024-01-26 19:17:53.230776: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M1 Pro
2024-01-26 19:17:53.230808: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 16.00 GB
2024-01-26 19:17:53.230821: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 5.33 GB
2024-01-26 19:17:53.231132: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2024-01-26 19:17:53.231545: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [9]:
model = keras.Sequential([
    keras.Input(shape=(IMG_SIZE, IMG_SIZE, 3)),
    # normalizer,
    dense_net,
    keras.layers.Flatten(),
    keras.layers.Dense(256, activation='relu'),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(2, activation="softmax"),
])

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 densenet169 (Functional)    (None, 7, 7, 1664)        12642880  
                                                                 
 flatten (Flatten)           (None, 81536)             0         
                                                                 
 dense (Dense)               (None, 256)               20873472  
                                                                 
 dropout (Dropout)           (None, 256)               0         
                                                                 
 dense_1 (Dense)             (None, 2)                 514       
                                                                 
Total params: 33516866 (127.86 MB)
Trainable params: 20873986 (79.63 MB)
Non-trainable params: 12642880 (48.23 MB)
_________________________________________________________________


In [10]:
# Compiling the model
model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

In [11]:
# Training the model
early_stopping = keras.callbacks.EarlyStopping(monitor='val_loss', patience=2, verbose=1)
mc = keras.callbacks.ModelCheckpoint('models/tb.h5', monitor='val_loss', save_best_only=True)

In [12]:
history = model.fit(train_generator, 
                    epochs=10, 
                    validation_data=val_generator, 
                    callbacks=[early_stopping, mc])

Epoch 1/10


2024-01-26 19:18:01.267049: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.


238/238 [==============================] - ETA: 0s - loss: 4.6119 - accuracy: 0.9376

/opt/miniconda3/envs/tensorflow/lib/python3.10/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


238/238 [==============================] - 67s 245ms/step - loss: 4.6119 - accuracy: 0.9376 - val_loss: 0.8669 - val_accuracy: 0.9794
Epoch 2/10
238/238 [==============================] - 49s 205ms/step - loss: 2.0971 - accuracy: 0.9687 - val_loss: 1.5754 - val_accuracy: 0.9666
Epoch 3/10
238/238 [==============================] - 50s 209ms/step - loss: 1.6127 - accuracy: 0.9750 - val_loss: 0.1677 - val_accuracy: 0.9923
Epoch 4/10
238/238 [==============================] - 49s 206ms/step - loss: 1.2034 - accuracy: 0.9760 - val_loss: 0.3036 - val_accuracy: 0.9906
Epoch 5/10
238/238 [==============================] - 51s 215ms/step - loss: 1.2126 - accuracy: 0.9779 - val_loss: 0.2339 - val_accuracy: 0.9906
Epoch 5: early stopping


In [13]:
# Evaluate the model on the test set
test_loss, test_acc = model.evaluate(test_generator)
print(f"Test Accuracy: {test_acc * 100:.2f}%")

73/73 [==============================] - 15s 208ms/step - loss: 0.5133 - accuracy: 0.9896
Test Accuracy: 98.96%


In [14]:
# Convert the model to TFLite format
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_model = converter.convert()


INFO:tensorflow:Assets written to: /var/folders/qt/hcj8mn3s4rq62dxlffdwnnzw0000gn/T/tmpv2fdrv0w/assets


INFO:tensorflow:Assets written to: /var/folders/qt/hcj8mn3s4rq62dxlffdwnnzw0000gn/T/tmpv2fdrv0w/assets
2024-01-26 19:23:11.202575: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:378] Ignored output_format.
2024-01-26 19:23:11.202597: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:381] Ignored drop_control_dependency.
2024-01-26 19:23:11.203727: I tensorflow/cc/saved_model/reader.cc:83] Reading SavedModel from: /var/folders/qt/hcj8mn3s4rq62dxlffdwnnzw0000gn/T/tmpv2fdrv0w
2024-01-26 19:23:11.251252: I tensorflow/cc/saved_model/reader.cc:51] Reading meta graph with tags { serve }
2024-01-26 19:23:11.251270: I tensorflow/cc/saved_model/reader.cc:146] Reading SavedModel debug info (if present) from: /var/folders/qt/hcj8mn3s4rq62dxlffdwnnzw0000gn/T/tmpv2fdrv0w
2024-01-26 19:23:11.344513: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:388] MLIR V1 optimization pass is not enabled
2024-01-26 19:23:11.389400: I tensorflow/cc/saved_model/load

In [15]:
# Save the TFLite model
with open('models/tb_model.tflite', 'wb') as f:
    f.write(tflite_model)

In [19]:
# Load the best model and evaluate on the test set
tb_model = keras.models.load_model("models/tb.h5")
tb_model.evaluate(test_generator)

73/73 [==============================] - 21s 236ms/step - loss: 0.5776 - accuracy: 0.9862


[0.5776378512382507, 0.9861949682235718]